## 베이스라인 모델

### import

In [1]:
import torch
import pickle
import cv2
import pandas as pd
import numpy as np
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score # ROC AUC 점수 계산 함수

import torch.nn as nn # 신경망 모듈
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스
from torch.utils.data import DataLoader # 데이터 로더 클래스

from efficientnet_pytorch import EfficientNet # EfficientNet 모델
from tqdm.notebook import tqdm # 진행률 표시 막대 
import multiprocessing

# 이미지 변환을 위한 모듈
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

### 시드값 고정

In [3]:
# 시드값 고정
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### GPU 장비 설정

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

### 데이터 준비

In [8]:
# 데이터 경로
with open('./data/pre_data/df_metainfo.pkl', 'rb') as f:
    data_dict = pickle.load(f)

person = data_dict['tree']

test = person[person['fold_branch_yn'] == 0]
train = person[person['fold_branch_yn'] != 0]
train

,id,branch_yn,root_yn,crown_yn,fruit_yn,gnarl_yn,loc,size,fold_branch_yn,fold_root_yn,fold_crown_yn,fold_fruit_yn,fold_gnarl_yn,fold_loc,fold_size
0,24_175_23002_tree,0,0,1,1,0,0,1,2,4,3,4,2,4,1
1,24_175_23003_tree,0,0,1,0,0,1,0,4,2,4,3,1,3,0
2,24_175_23006_tree,0,0,1,0,0,1,0,2,2,1,2,0,3,0
3,24_175_23009_tree,1,0,0,0,0,0,0,2,3,2,4,3,4,0
4,24_175_23010_tree,0,0,1,1,1,1,2,4,0,0,3,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,5_29_23037_tree,0,0,1,0,0,1,0,1,0,0,3,2,3,1
988,5_29_23038_tree,0,1,1,0,0,1,1,1,0,2,0,3,0,2
992,5_29_23043_tree,0,1,1,1,0,1,1,1,0,2,2,3,3,2
993,5_29_23044_tree,1,1,1,1,0,1,1,1,1,3,1,1,3,4


In [9]:

test = test.iloc[:, :1]
train = train[['id', 'branch_yn', 'root_yn', 'crown_yn', 'fruit_yn', 'gnarl_yn']]
train


,id,branch_yn,root_yn,crown_yn,fruit_yn,gnarl_yn
0,24_175_23002_tree,0,0,1,1,0
1,24_175_23003_tree,0,0,1,0,0
2,24_175_23006_tree,0,0,1,0,0
3,24_175_23009_tree,1,0,0,0,0
4,24_175_23010_tree,0,0,1,1,1
...,...,...,...,...,...,...
987,5_29_23037_tree,0,0,1,0,0
988,5_29_23038_tree,0,1,1,0,0
992,5_29_23043_tree,0,1,1,1,0
993,5_29_23044_tree,1,1,1,1,0


In [10]:
train.iloc[:, 1:] = train.iloc[:, 1:].values.astype('int64')

c:\Users\USER\anaconda3\envs\torch\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 796 entries, 0 to 994
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         796 non-null    object
 1   branch_yn  796 non-null    int64 
 2   root_yn    796 non-null    int64 
 3   crown_yn   796 non-null    int64 
 4   fruit_yn   796 non-null    int64 
 5   gnarl_yn   796 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 43.5+ KB


In [12]:
train.head()

,id,branch_yn,root_yn,crown_yn,fruit_yn,gnarl_yn
0,24_175_23002_tree,0,0,1,1,0
1,24_175_23003_tree,0,0,1,0,0
2,24_175_23006_tree,0,0,1,0,0
3,24_175_23009_tree,1,0,0,0,0
4,24_175_23010_tree,0,0,1,1,1


In [13]:
test.head()

,id
9,24_175_23018_tree
13,24_176_23021_tree
14,24_176_23022_tree
15,24_176_23023_tree
22,24_176_23032_tree


- 훈련 데이터, 검증 데이터 분리

In [23]:
# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(train,
                                test_size=0.1,
                                stratify=train[['root_yn','crown_yn','gnarl_yn']],
                                random_state=42)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716 entries, 0 to 715
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         716 non-null    object
 1   branch_yn  716 non-null    int64 
 2   root_yn    716 non-null    int64 
 3   crown_yn   716 non-null    int64 
 4   fruit_yn   716 non-null    int64 
 5   gnarl_yn   716 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 33.7+ KB


In [25]:
train["fruit_yn"].value_counts()

0    406
1    310
Name: fruit_yn, dtype: int64

In [26]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         80 non-null     object
 1   branch_yn  80 non-null     int64 
 2   root_yn    80 non-null     int64 
 3   crown_yn   80 non-null     int64 
 4   fruit_yn   80 non-null     int64 
 5   gnarl_yn   80 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 3.9+ KB


In [28]:
valid["fruit_yn"].value_counts()

1    41
0    39
Name: fruit_yn, dtype: int64

- 데이터셋 클래스 정의

In [29]:
class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출
        # 전달받은 인수 저장
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    # 데이터셋 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]             # 이미지 ID
        img_path = self.img_dir + img_id + '.jpg' # 2. 이미지 파일 경로
        image = cv2.imread(img_path)              # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        # 이미지 변환 
        if self.transform is not None:
            image = self.transform(image=image)['image']  # 3.
        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃값도 반환 
        if self.is_test:  # 4.
            return image # 5. 테스트용일 때
        else:
            # # 타깃값 n개 중 가장 큰 값의 인덱스  # 6.
            # label = np.argmax(self.df.iloc[idx, 1:])
            # pandas indexing 버그인지 모르겠으나, 특정열을 선택하면 obejct로 바뀌어 버려서 다시 int로 재변환
            label = self.df.iloc[idx, 1:].values.astype("int64")
            return image, label # 훈련/검증용일 때  # 7.

- 이미지 변환기 정의

In [30]:
# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(224, 224),       # 1. 이미지 크기 조절 
    # A.RandomBrightnessContrast(brightness_limit=0.2, # 2. 밝기 대비 조절
    #                            contrast_limit=0.2, p=0.3),
    # A.VerticalFlip(p = 0.2),    # 상하 대칭 변환
    # A.HorizontalFlip(p = 0.5),  # 좌우 대칭 변환 
    # A.ShiftScaleRotate(       # 3. 이동, 스케일링, 회전 변환
    #     shift_limit = 0.1,
    #     scale_limit = 0.2,
    #     rotate_limit = 30, p = 0.3),
    # A.OneOf([A.Emboss(p = 1),   # 4. 양각화, 날카로움, 블러 효과
    #          A.Sharpen(p = 1),
    #          A.Blur(p = 1)], p = 0.3),
    # A.PiecewiseAffine(p = 0.3), # 5. 어파인 변환 
    A.Normalize(),            # 6. 정규화 변환 
    ToTensorV2()              # 7. 텐서로 변환
])

In [31]:
# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(224, 224), # 이미지 크기 조절 
    A.Normalize(),      # 정규화 변환
    ToTensorV2()        # 텐서로 변환
])

- 데이터 셋 및 데이터 로더 생성

In [84]:
# def seed_worker(worker_id):   #데이터 로더 시드값 고정 함수
#     worker_seed = torch.initial_seed() % 2**32
#     np.random.seed(worker_seed)
#     random.seed(worker_seed)
    
# g = torch.Generator()  # 제너레이터 생성
# g.manual_seed(0)  # 제너레이터 시드값 고정

In [32]:
img_dir = './image/crop_data/tree/'
batch_size = 4

loader_train = DataLoader(
    ImageDataset(train, img_dir=img_dir, transform=transform_train, is_test=False),
    batch_size=batch_size, shuffle=True,
)
loader_valid = DataLoader(
    ImageDataset(valid, img_dir=img_dir, transform=transform_test, is_test=False),
    batch_size=batch_size, shuffle=False,
)

In [33]:
for batch in loader_train:
    print(batch[0].shape)
    print(batch[1].shape)
    break

torch.Size([4, 3, 224, 224])
torch.Size([4, 5])


### 모델 생성

- EfficientNet 모델 생성

In [34]:
# 사전 훈련된 efficientnet-b7 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=10) 
model = model.to(device) # 장비 할당

Loaded pretrained weights for efficientnet-b4


### 모델 훈련 및 성능 검증

- 손실함수와 옵티마이저 설정

In [35]:
# 손실 함수
criterion = nn.CrossEntropyLoss()
# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

- 훈련 및 성능 검증

In [89]:
epochs = 10
model_save_path = "./model/"

# 총 에폭만큼 반복
for epoch in range(epochs):
    # <훈련>
    model.train()        # 모델을 훈련 상태로 설정 
    epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
    
    # '반복 횟수'만큼 반복 
    for images, labels in tqdm(loader_train):
        # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        # 각 target에 대한 loss 계산
        # loss = criterion(outputs, labels)
        loss1 = criterion(outputs[:, 0:2], labels[:, 0])
        loss2 = criterion(outputs[:, 2:4], labels[:, 1])
        loss3 = criterion(outputs[:, 4:6], labels[:, 2])
        loss4 = criterion(outputs[:, 6:8], labels[:, 3])
        loss5 = criterion(outputs[:, 8:10], labels[:, 4])
        # 최종 loss 합산
        final_loss = loss1 + loss2 + loss3 + loss4 + loss5
        # 현재 배치에서의 손실 추가 (훈련 데이터용)
        epoch_train_loss += final_loss.item()
        final_loss.backward() # 역전파 수행
        optimizer.step() # 가중치 갱신
        # break

    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')

    # break
    # <검증>
    model.eval()          # 모델을 평가 상태로 설정 
    epoch_valid_loss = 0  # 에폭별 손실값 초기화 (검증 데이터용)
    pred_array = {
        "branch_yn": [],
        "root_yn": [],
        "crown_yn": [],
        "fruit_yn": [],
        "gnarl_yn": []
    }
    true_array = {
        "branch_yn": [],
        "root_yn": [],
        "crown_yn": [],
        "fruit_yn": [],
        "gnarl_yn": []
    }
    with torch.no_grad(): # 기울기 계산 비활성화
        # 미니배치 단위로 검증
        for images, labels in tqdm(loader_valid):
            images = images.to(device)
            labels = labels.to(device)
            # 출력값 게산
            outputs = model(images)
            # 각 target에 대한 loss 계산
            loss1 = criterion(outputs[:, 0:2], labels[:, 0])
            loss2 = criterion(outputs[:, 2:4], labels[:, 1])
            loss3 = criterion(outputs[:, 4:6], labels[:, 2])
            loss4 = criterion(outputs[:, 6:8], labels[:, 3])
            loss4 = criterion(outputs[:, 8:10], labels[:, 4])
            # 최종 loss 합산
            final_loss = loss1 + loss2 + loss3 + loss4
            epoch_valid_loss += final_loss.item()
            # metric 계산
            pred_array["branch_yn"].extend(outputs[:, 0:2].argmax(dim=-1).tolist())
            pred_array["root_yn"].extend(outputs[:, 2:4].argmax(dim=-1).tolist())
            pred_array["crown_yn"].extend(outputs[:, 4:6].argmax(dim=-1).tolist())
            pred_array["fruit_yn"].extend(outputs[:, 6:8].argmax(dim=-1).tolist())
            pred_array["gnarl_yn"].extend(outputs[:, 8:10].argmax(dim=-1).tolist())
            true_array["branch_yn"].extend(labels[:, 0].tolist())
            true_array["root_yn"].extend(labels[:, 1].tolist())
            true_array["crown_yn"].extend(labels[:, 2].tolist())
            true_array["fruit_yn"].extend(labels[:, 3].tolist())
            true_array["gnarl_yn"].extend(labels[:, 4].tolist())
            # break
        
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')

    # 정확도, F1 점수 계산 (target별 따로 게산 후 산술평균)
    accuracy = np.mean([accuracy_score(a, p) for a, p in zip(true_array.values(), pred_array.values())])
    f1 = np.mean([f1_score(a, p, average='macro') for a, p in zip(true_array.values(), pred_array.values())])

    print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 정확도: {accuracy:.4f} / 검증 데이터 F1 점수: {f1:.4f}')
    # 에폭마다 모델 저장
    torch.save(model.state_dict(), f"{model_save_path}tree_model{epoch+1}_loss{epoch_valid_loss/len(loader_valid):.4f}_acc{accuracy:.4f}_f1{f1:.4f}.pth")
    print(f"에폭 {epoch+1}의 모델이 {model_save_path}tree_model{epoch+1}_loss{epoch_valid_loss/len(loader_valid):.4f}_acc{accuracy:.4f}_f1{f1:.4f}.pth로 저장되었습니다.")

    # break

  0%|          | 0/179 [00:00<?, ?it/s]

에폭 [1/10] - 훈련 데이터 손실값 : 1.5527


  0%|          | 0/20 [00:00<?, ?it/s]

에폭 [1/10] - 검증 데이터 손실값 : 0.9959
에폭 [1/10] - 검증 데이터 정확도: 0.9625 / 검증 데이터 F1 점수: 0.4904
에폭 1의 모델이 ./model/person_model1_loss0.9959_acc0.9625_f10.4904.pth로 저장되었습니다.


  0%|          | 0/179 [00:00<?, ?it/s]

...

에폭 [9/10] - 검증 데이터 손실값 : 0.7397
에폭 [9/10] - 검증 데이터 정확도: 0.9375 / 검증 데이터 F1 점수: 0.5192
에폭 9의 모델이 ./model/person_model9_loss0.7397_acc0.9375_f10.5192.pth로 저장되었습니다.


  0%|          | 0/179 [00:00<?, ?it/s]

에폭 [10/10] - 훈련 데이터 손실값 : 0.1575


  0%|          | 0/20 [00:00<?, ?it/s]

에폭 [10/10] - 검증 데이터 손실값 : 0.7605
에폭 [10/10] - 검증 데이터 정확도: 0.9312 / 검증 데이터 F1 점수: 0.5047
에폭 10의 모델이 ./model/person_model10_loss0.7605_acc0.9312_f10.5047.pth로 저장되었습니다.


### 예측 결과

- 테스트용 데이터셋

In [48]:
loader_test = DataLoader(
    ImageDataset(test, img_dir=img_dir, transform=transform_test, is_test=True),
    batch_size=batch_size, shuffle=False
)

- 예측

In [49]:
model.eval() # 모델을 평가 상태로 설정 
preds = {
    "branch_yn": [],
    "root_yn": [],
    "crown_yn": [],
    "fruit_yn": [],
    "gnarl_yn": []
}

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        # 타깃 예측 확률 
        preds["branch_yn"].extend(outputs[:, 0:2].argmax(dim=-1).tolist())
        preds["root_yn"].extend(outputs[:, 2:4].argmax(dim=-1).tolist())
        preds["crown_yn"].extend(outputs[:, 4:6].argmax(dim=-1).tolist())
        preds["fruit_yn"].extend(outputs[:, 6:8].argmax(dim=-1).tolist())
        preds["gnarl_yn"].extend(outputs[:, 8:10].argmax(dim=-1).tolist())

In [61]:
for v in preds.values():
    print("example")
    print(v[:12])

example
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
example
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
example
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1]


- 결과 저장

In [65]:
# submission[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds
# submission.to_csv('./data/submission.csv', index=False)

In [ ]:
# # 원래 형태로 되돌리기
# original_train = encoded_train.copy()

# columns_to_convert = ['door_yn', 'roof_yn', 'window_cnt']
# for col in columns_to_convert:
#     # 각 원-핫 인코딩된 컬럼을 복원
#     original_train[col] = original_train.loc[:, original_train.columns.str.startswith(col)].idxmax(axis=1)
#     # 복원된 컬럼의 레이블 추출 (col_value에서 "col_" 뒤의 문자열 추출)
#     original_train[col] = original_train[col].apply(lambda x: x.split('_')[2])
#     # 원-핫 인코딩된 컬럼 삭제
#     original_train.drop(columns=[c for c in original_train.columns if c.startswith(f"{col}_")], inplace=True)